# PART_02. Напишем django app приложение.

### Написание вашего первого приложения Django, часть 2

Этот урок начинается с того места, где закончился урок 1. 

Мы настроим базу данных, создадим вашу первую модель и кратко познакомимся с автоматически создаваемым административным сайтом Django.

`Где получить помощь:`

Если у вас возникли проблемы с изучением этого руководства, перейдите в раздел «Помощь» в разделе «Часто задаваемые вопросы».

### Настройка базы данных

Теперь откройте __`mysite/settings.py`__. Это обычный модуль Python с переменными уровня модуля, представляющими настройки Django.

По умолчанию в конфигурации используется __`SQLite`__. Если вы новичок в работе с базами данных или просто хотите попробовать Django, это самый простой выбор. SQLite включен в Python, поэтому вам не нужно будет устанавливать что-либо еще для поддержки вашей базы данных. Однако при запуске вашего первого реального проекта вы можете захотеть использовать более масштабируемую базу данных, такую как __`PostgreSQL`__, чтобы избежать головной боли при переключении баз данных в будущем.

Если вы хотите использовать другую базу данных, установите соответствующие привязки к базе данных и измените следующие ключи в элементе __`DATABASES «default»`__, чтобы они соответствовали настройкам подключения к вашей базе данных:


__`ENGINE`__:
- «django.db.backends.sqlite3», 
- «django.db.backends.postgresql», 
- «django.db.backends.mysql», 
- «django.db.backends.oracle». 
- Другие бэкэнды также доступны.

__`NAME`__ :
- Имя вашей базы данных. Если вы используете SQLite, база данных будет представлять собой файл на вашем компьютере; в этом случае NAME должен быть полным абсолютным путем, включая имя файла, к этому файлу. Значение по умолчанию _BASE_DIR/'db.sqlite3'_ сохранит файл в каталоге вашего проекта.

Если вы не используете `SQLite` в качестве базы данных, необходимо добавить дополнительные настройки, такие как `USER`, `PASSWORD` и `HOST`. Более подробную информацию см. в справочной документации по БАЗАМ ДАННЫХ.

### Для баз данных, отличных от SQLite

Если вы используете базу данных помимо `SQLite`, убедитесь, что к этому моменту вы создали базу данных. Сделайте это с помощью __`«CREATE DATABASE имя_базы_данных;»`__ в интерактивной подсказке вашей базы данных.

Также убедитесь, что пользователь базы данных, указанный в __`mysite/settings.py`__, имеет права «создать базу данных». Это позволяет автоматически создать тестовую базу данных, которая понадобится в следующем руководстве.

Если вы используете SQLite, вам не нужно ничего создавать заранее — файл базы данных будет создан автоматически, когда он понадобится.

Пока вы редактируете файл __`mysite/settings.py`__, установите __`TIME_ZONE`__ в свой часовой пояс.

Также обратите внимание на параметр __`INSTALLED_APPS`__ в верхней части файла. Здесь хранятся имена всех приложений Django, которые активированы в этом экземпляре Django. Приложения можно использовать в нескольких проектах, их можно упаковывать и распространять для использования другими в своих проектах.

По умолчанию __`INSTALLED_APPS`__ содержит следующие приложения, все из которых поставляются с Django:

- django.contrib.admin – сайт администрирования. Вы скоро им воспользуетесь.
- django.contrib.auth — система аутентификации.
- django.contrib.contenttypes – платформа для типов контента.
- django.contrib.sessions – инфраструктура сеансов.
- django.contrib.messages – платформа обмена сообщениями.
- django.contrib.staticfiles – платформа для управления статическими файлами.

Эти приложения включены по умолчанию для удобства в обычном случае.

Однако некоторые из этих приложений используют по крайней мере одну таблицу базы данных, поэтому нам необходимо создать таблицы в базе данных, прежде чем мы сможем их использовать. Для этого выполните следующую команду:

In [ ]:
$ python manage.py migrate

Команда миграции просматривает параметр __`INSTALLED_APPS`__ и создает все необходимые таблицы базы данных в соответствии с настройками базы данных в вашем файле __`mysite/settings.py`__ и миграциями базы данных, поставляемыми вместе с приложением (мы рассмотрим их позже). Вы увидите сообщение для каждой примененной миграции. Если вам интересно, запустите клиент командной строки для вашей базы данных и введите:
- __\dt__ `(PostgreSQL)`, 
- __ПОКАЗАТЬ ТАБЛИЦЫ;__ `(MariaDB, MySQL)`, 
- __.tables__ `(SQLite)` 
- __SELECT TABLE_NAME FROM USER_TABLES;__ `(Oracle)` 

для отображения таблиц, созданных Django.

### Для минималистов

Как мы уже говорили выше, стандартные приложения включены для общего случая, но они нужны не всем. Если вам не нужны некоторые из них или все, не стесняйтесь закомментировать или удалить соответствующие строки из __`INSTALLED_APPS`__ перед запуском миграции. Команда миграции выполнит миграцию только для приложений в __`INSTALLED_APPS`__.

# Создание моделей

Теперь мы определим ваши модели — по сути, структуру вашей базы данных с дополнительными метаданными.

`Философия`

Модель — это единственный и точный источник информации о ваших данных. Он содержит основные поля и поведение данных, которые вы храните. Django следует принципу __`DRY`__. Цель состоит в том, чтобы определить вашу модель данных в одном месте и автоматически извлекать из нее данные.

Это включает в себя миграции — в отличие, например, от __`Ruby On Rails`__, миграции полностью извлекаются из файла вашей модели и, по сути, представляют собой историю, которую Django может просмотреть, чтобы обновить схему вашей базы данных в соответствии с вашими текущими моделями.

В нашем приложении для опроса мы создадим две модели: 
1) вопрос 
2) выбор

- __`Вопрос`__: имеет вопрос и дату публикации. 
- __`Выбор`__: имеет два поля: 1)текст выбора, 2)подсчет голосов. Каждый вариант выбора связан с вопросом.

Эти концепции представлены классами Python. Отредактируйте файл __`polls/models.py`__, чтобы он выглядел следующим образом:

In [ ]:
# mysite/polls/models.py

from django.db import models


class Question(models.Model):
    question_text = models.CharField(max_length=200)
    pub_date = models.DateTimeField("date published")


class Choice(models.Model):
    question = models.ForeignKey(Question, on_delete=models.CASCADE)
    choice_text = models.CharField(max_length=200)
    votes = models.IntegerField(default=0)

Здесь каждая модель представлена классом, который является подклассом __`django.db.models.Model`__. Каждая модель имеет несколько переменных класса, каждая из которых представляет поле базы данных в модели.

Каждое поле представлено экземпляром класса __`Field`__ — например.

- __`CharField`__ для символьных полей 
- __`DateTimeField`__ для дат и времени. 

Это сообщает Django, какой тип данных содержит каждое поле.

Имя каждого экземпляра __`Field`__ (например, questions_text или pub_date) — это имя поля в машиночитаемом формате. Вы будете использовать это значение в своем коде Python, а ваша база данных будет использовать его в качестве имени столбца.

Вы можете использовать необязательный первый позиционный аргумент __`Field`__, чтобы обозначить удобочитаемое имя. Он используется в нескольких интроспективных частях Django и используется в качестве документации. Если это поле не указано, Django будет использовать машиночитаемое имя. В этом примере мы определили только удобочитаемое имя для __`Question.pub_date`__. Для всех остальных полей в этой модели машиночитаемого имени поля будет достаточно в качестве удобочитаемого имени.

Некоторые классы __`Field`__ имеют обязательные аргументы. Например, __`CharField`__ требует, чтобы вы указали `max_length`. Как мы скоро увидим, это используется не только в схеме базы данных, но и при проверке.

__`Field`__ также может иметь различные необязательные аргументы; в данном случае мы установили значение `votes` по `default` равное 0.

Наконец, обратите внимание, что связь определяется с использованием __`ForeignKey`__. Это говорит Django, что каждый вариант ответа (Choice) связан с одним вопросом (Question). Django поддерживает все распространенные отношения базы данных:
- «многие-к-одному», 
- «многие-ко-многим» 
- «один-к-одному».

### Активация моделей

Этот небольшой фрагмент кода модели дает Django много информации. С его помощью Django может:

- Создайте схему базы данных (операторы CREATE TABLE) для этого приложения.
- Создайте API доступа к базе данных Python для доступа к объектам вопросов и вариантов выбора.

Но сначала нам нужно сообщить нашему проекту, что приложение для опросов установлено.

`Философия`

Приложения Django являются «подключаемыми»: вы можете использовать приложение в нескольких проектах и распространять приложения, поскольку их не обязательно привязывать к конкретной установке Django.

Чтобы включить приложение в наш проект, нам нужно добавить ссылку на его класс конфигурации в параметр __`INSTALLED_APPS`__. Класс __`PollsConfig`__ находится в файле __`polls/apps.py`__, поэтому его пунктирный путь — __`polls.apps.PollsConfig`__.

Отредактируйте файл __`mysite/settings.py`__ и добавьте этот пунктирный путь к параметру __`INSTALLED_APPS`__. Это будет выглядеть так:

In [ ]:
# mysite/mysite/settings.py

INSTALLED_APPS = [
    "polls.apps.PollsConfig",
    "django.contrib.admin",
    "django.contrib.auth",
    "django.contrib.contenttypes",
    "django.contrib.sessions",
    "django.contrib.messages",
    "django.contrib.staticfiles",
]

Теперь Django знает, что нужно включить приложение (app) для опросов (polls). Давайте запустим еще одну команду:

In [ ]:
$ python manage.py makemigrations polls

Вы должны увидеть что-то похожее на следующее:

```
Migrations for 'polls':
  polls/migrations/0001_initial.py
    - Create model Question
    - Create model Choice
```
Запуская __`makemigrations`__, вы сообщаете Django, что внесли некоторые изменения в свои модели (в данном случае вы внесли новые) и хотите, чтобы эти изменения были сохранены в виде миграции.

Миграции — это то, как Django сохраняет изменения в ваших моделях (и, следовательно, в схеме вашей базы данных) — это файлы на диске. Если хотите, вы можете прочитать миграцию для вашей новой модели; это файл __`polls/migrations/0001_initial.py`__. Не волнуйтесь, от вас не ожидается, что вы будете читать их каждый раз, когда Django создает их, но они созданы для того, чтобы их можно было редактировать человеком на случай, если вы захотите вручную настроить, как Django что-то меняет.

Есть команда, которая будет запускать миграцию за вас и автоматически управлять схемой вашей базы данных — она называется __`migrate`__, и мы к ней скоро подойдем — но сначала давайте посмотрим, какой SQL будет выполняться при этой миграции. Команда __`sqlmigrate`__ принимает имена миграции и возвращает их SQL:

In [ ]:
$ python manage.py sqlmigrate polls 0001

Вы должны увидеть что-то похожее на следующее (мы переформатировали его для удобства чтения):

In [ ]:
BEGIN;
--
-- Create model Question
--
CREATE TABLE "polls_question" (
    "id" bigint NOT NULL PRIMARY KEY GENERATED BY DEFAULT AS IDENTITY,
    "question_text" varchar(200) NOT NULL,
    "pub_date" timestamp with time zone NOT NULL
);
--
-- Create model Choice
--
CREATE TABLE "polls_choice" (
    "id" bigint NOT NULL PRIMARY KEY GENERATED BY DEFAULT AS IDENTITY,
    "choice_text" varchar(200) NOT NULL,
    "votes" integer NOT NULL,
    "question_id" bigint NOT NULL
);
ALTER TABLE "polls_choice"
  ADD CONSTRAINT "polls_choice_question_id_c5b4b260_fk_polls_question_id"
    FOREIGN KEY ("question_id")
    REFERENCES "polls_question" ("id")
    DEFERRABLE INITIALLY DEFERRED;
CREATE INDEX "polls_choice_question_id_c5b4b260" ON "polls_choice" ("question_id");

COMMIT;

Обратите внимание на следующее:

- Точный результат будет зависеть от используемой вами базы данных. Приведенный выше пример создан для PostgreSQL.
- Названия таблиц генерируются автоматически путем объединения названия приложения (опросов) и названия модели в нижнем регистре – вопрос и выбор. (Вы можете переопределить это поведение.)
- Первичные ключи (ID) добавляются автоматически. (Вы также можете переопределить это.)
- По соглашению Django добавляет «_id» к имени поля внешнего ключа. (Да, вы также можете переопределить это.)
- Отношения внешнего ключа явно определяются ограничением FOREIGN KEY. Не беспокойтесь об ОТЛОЖЕННЫХ частях; он сообщает PostgreSQL не применять внешний ключ до конца транзакции.
- Он адаптирован для используемой вами базы данных, поэтому типы полей, специфичные для базы данных, такие как auto_increment (MySQL), bigint PRIMARY KEY, GENERATED BY DEFAULT AS IDENTITY (PostgreSQL) или автоинкремент целочисленного первичного ключа (SQLite), обрабатываются автоматически. То же самое касается заключения имен полей в кавычки – например, с использованием двойных или одинарных кавычек.
- Команда sqlmigrate на самом деле не запускает миграцию в вашей базе данных — вместо этого она выводит ее на экран, чтобы вы могли увидеть, что, по мнению SQL Django, требуется. Это полезно для проверки того, что собирается делать Django, или если у вас есть администраторы баз данных, которым для внесения изменений требуются сценарии SQL.

Если вам интересно, вы также можете запустить проверку python Manage.py; это проверяет наличие проблем в вашем проекте, не делая миграции и не затрагивая базу данных.

Теперь снова запустите миграцию, чтобы создать эти таблицы моделей в вашей базе данных:

In [ ]:
$ python manage.py migrate

```
Operations to perform:
  Apply all migrations: admin, auth, contenttypes, polls, sessions
Running migrations:
  Rendering model states... DONE
  Applying polls.0001_initial... OK
```

Команда миграции берет все миграции, которые не были применены (Django отслеживает, какие из них были применены, с помощью специальной таблицы в вашей базе данных, называемой `django_migrations`) и запускает их в вашей базе данных - по сути, синхронизируя изменения, внесенные вами в ваши модели, со схемой. в базе данных.

Миграции очень эффективны и позволяют вам изменять свои модели с течением времени по мере разработки проекта без необходимости удалять базу данных или таблицы и создавать новые — она специализируется на обновлении вашей базы данных в реальном времени без потери данных. Мы рассмотрим их более подробно в более поздней части руководства, а пока запомните трехэтапное руководство по внесению изменений в модель:

- Измените свои модели (__`в models.py`__).
- Запустите __`python Manage.py makemigrations`__, чтобы создать миграцию для этих изменений.
- Запустите __`python Manage.py Migrate`__, чтобы применить эти изменения к базе данных.

Причина, по которой существуют отдельные команды для создания и применения миграций, заключается в том, что вы фиксируете миграции в своей системе контроля версий и отправляете их вместе со своим приложением; они не только упрощают вашу разработку, но и могут использоваться другими разработчиками и в производстве.

Прочтите документацию __`django-admin`__ для получения полной информации о возможностях утилиты __`Manage.py`__.

# Играем с API

Теперь давайте перейдем к интерактивной оболочке Python и поиграем с бесплатным API, который предоставляет вам Django. Чтобы вызвать оболочку Python, используйте следующую команду:

In [ ]:
$ python manage.py shell

Мы используем это вместо того, чтобы просто вводить «python», потому что __`Manage.py`__ устанавливает переменную среды __`DJANGO_SETTINGS_MODULE`__, которая дает Django путь импорта Python к вашему файлу __`mysite/settings.py`__.

Зайдя в оболочку, изучите API базы данных:

In [ ]:
>>> from polls.models import Choice, Question  # Импортируйте только что написанные классы моделей.

# В системе пока нет вопросов.
>>> Question.objects.all()
-->
<QuerySet []>


# Создайте новый вопрос.
# Поддержка часовых поясов включена в файле настроек по умолчанию, поэтому
# Django ожидает дату и время с помощью tzinfo для pub_date. Используйте timezone.now()
# вместо datetime.datetime.now() и все будет правильно.
>>> from django.utils import timezone
>>> q = Question(question_text="What's new?", pub_date=timezone.now())

# Сохраняем объект в базу данных. Вам нужно явно вызвать save().
>>> q.save()

# Теперь у него есть идентификатор.
>>> q.id
-->
1

# Доступ к значениям полей модели через атрибуты Python.
>>> q.question_text
--> "What's new?"

>>> q.pub_date
--> datetime.datetime(2012, 2, 26, 13, 0, 0, 775217, tzinfo=datetime.timezone.utc)

# Измените значения, изменив атрибуты, а затем вызвав save().
>>> q.question_text = "What's up?"
>>> q.save()

#objects.all() отображает все вопросы в базе данных.
>>> Question.objects.all()
--> <QuerySet [<Question: Question object (1)>]>

Подождите минуту. `<Question: Question object (1)>`  не является полезным представлением этого объекта. Давайте исправим это, отредактировав модель вопроса (в файле __`polls/models.py`__) и добавив метод __`__str__()`__ как к вопросу, так и к выбору:

In [ ]:
# mysite/polls/models.py

from django.db import models


class Question(models.Model):
    # ...
    def __str__(self):
        return self.question_text


class Choice(models.Model):
    # ...
    def __str__(self):
        return self.choice_text

Важно добавлять методы __`__str__()`__ в ваши модели не только для вашего удобства при работе с интерактивной подсказкой, но и потому, что представления объектов используются во всей автоматически создаваемой административной панели Django.

Давайте также добавим в эту модель собственный метод:

In [ ]:
# mysite/polls/models.py

import datetime

from django.db import models
from django.utils import timezone


class Question(models.Model):
    # ...
    def was_published_recently(self):
        return self.pub_date >= timezone.now() - datetime.timedelta(days=1)

Обратите внимание на добавление __`import datetime`__ и __`from django.utils import timezone`__ для ссылки на стандартный модуль __`Python datetime`__ и утилиты Django, связанные с часовым поясом, в __`django.utils.timezone`__ соответственно. Если вы не знакомы с обработкой часовых поясов в Python, вы можете узнать больше в документации по поддержке часовых поясов.

Сохраните эти изменения и запустите новую интерактивную оболочку Python, снова запустив оболочку __`python Manage.py`__:

In [ ]:
$ python manage.py shell

In [ ]:
>>> from polls.models import Choice, Question


# Убедитесь, что добавление __str__() сработало.
>>> Question.objects.all()
--> <QuerySet [<Question: What's up?>]>

# Django предоставляет богатый API поиска в базе данных, полностью управляемый
# аргументы ключевого слова.
>>> Question.objects.filter(id=1)
--> <QuerySet [<Question: What's up?>]>
           
>>> Question.objects.filter(question_text__startswith="What")
--> <QuerySet [<Question: What's up?>]>

# Получите вопрос, опубликованный в этом году.
>>> from django.utils import timezone
>>> current_year = timezone.now().year
>>> Question.objects.get(pub_date__year=current_year)
--> <Question: What's up?>

# Запросите несуществующий идентификатор, это вызовет исключение.
>>> Question.objects.get(id=2)
--> Traceback (most recent call last):
    ...
DoesNotExist: Question matching query does not exist.

# Поиск по первичному ключу является наиболее распространенным случаем, поэтому Django предоставляет
# ярлык для точного поиска по первичному ключу.
# Следующий код идентичен 
>>> Question.objects.get(pk=1)
--> <Question: What's up?>

# Убедитесь, что наш собственный метод работает.
>>> q = Question.objects.get(pk=1)
>>> q.was_published_recently()
--> True

# Дайте вопросу несколько вариантов выбора. Вызов create создает новый
# Объект выбора, выполняет оператор INSERT, добавляет выбор в набор
# доступных вариантов выбора и возвращает новый объект Choice. Джанго создает
# набор для хранения «другой стороны» отношения ForeignKey
# (например, выбор вопроса), доступ к которому можно получить через API.
>>> q = Question.objects.get(pk=1)

# Отображение любых вариантов из набора связанных объектов — пока ничего.
>>> q.choice_set.all()
--> <QuerySet []>

# Создайте три варианта выбора.
>>> q.choice_set.create(choice_text="Not much", votes=0)
--> <Choice: Not much>
>>> q.choice_set.create(choice_text="The sky", votes=0)
--> <Choice: The sky>
>>> c = q.choice_set.create(choice_text="Just hacking again", votes=0)

# Объекты выбора имеют доступ API к связанным с ними объектам вопросов.
>>> c.question
--> <Question: What's up?>

# И наоборот: объекты вопросов получают доступ к объектам выбора.
>>> q.choice_set.all()
--> <QuerySet [<Choice: Not much>, <Choice: The sky>, <Choice: Just hacking again>]>
>>> q.choice_set.count()
--> 3

# API автоматически отслеживает отношения настолько, насколько вам нужно.
# Используйте двойное подчеркивание для разделения отношений.
# Это работает на любом уровне глубины; нет предела.
# Найти все варианты ответа на любой вопрос, дата публикации которого приходится на этот год.
# (повторное использование переменной current_year, которую мы создали выше).
>>> Choice.objects.filter(question__pub_date__year=current_year)
--> <QuerySet [<Choice: Not much>, <Choice: The sky>, <Choice: Just hacking again>]>

# Давайте удалим один из вариантов. Используйте для этого delete().
>>> c = q.choice_set.filter(choice_text__startswith="Just hacking")
>>> c.delete()

Дополнительные сведения об отношениях модели см. в разделе Доступ к связанным объектам. Дополнительные сведения о том, как использовать двойное подчеркивание для поиска полей через API, см. в разделе Поиск полей. Полную информацию об API базы данных см. в нашем справочнике по API базы данных.

# Знакомство с администратором Django

`Философия`

Создание сайтов администрирования, на которых ваши сотрудники или клиенты могут добавлять, изменять и удалять контент, — это утомительная работа, не требующая особого творчества. По этой причине Django полностью автоматизирует создание интерфейсов администратора для моделей.

Django был написан в среде редакции с очень четким разделением между «издателями контента» и «публичным» сайтом. Менеджеры сайтов используют систему для добавления новостей, событий, спортивных результатов и т. д., и этот контент отображается на общедоступном сайте. Django решает задачу создания единого интерфейса для администраторов сайта для редактирования контента.

Администратор не предназначен для использования посетителями сайта. Это для администраторов сайтов.

# Создание администратора

Сначала нам нужно создать пользователя, который сможет войти на сайт администратора. Выполните следующую команду:

In [ ]:
$ python manage.py createsuperuser

Введите желаемое имя пользователя и нажмите Enter.
```
Имя пользователя: admin
```
Затем вам будет предложено ввести желаемый адрес электронной почты:
```
Адрес электронной почты: admin@example.com
```
Последний шаг — ввести пароль. Вам будет предложено ввести пароль дважды, второй раз в качестве подтверждения первого.
```
Пароль: **********
Пароль еще раз): *********
```
Суперпользователь успешно создан.

# Запустите сервер разработки

Сайт администрирования Django активирован по умолчанию. Давайте запустим сервер разработки и изучим его.

Если сервер не запущен, запустите его так:

In [ ]:
$ python manage.py runserver

Теперь откройте веб-браузер и перейдите в __`«/admin/»`__ в своем локальном домене – например, __`http://127.0.0.1:8000/admin/`__. Вы должны увидеть экран входа администратора:


Поскольку перевод включен по умолчанию, если вы установите __`LANGUAGE_CODE`__, экран входа в систему будет отображаться на заданном языке (если в Django есть соответствующие переводы).

Теперь попробуйте войти в систему с учетной записью суперпользователя, которую вы создали на предыдущем шаге. Вы должны увидеть индексную страницу администратора Django:

Вы должны увидеть несколько типов редактируемого контента: 
- группы и пользователей. 

Они предоставляются __`django.contrib.auth`__, инфраструктурой аутентификации, поставляемой Django.

Но где наше приложение для опросов? Он не отображается на главной странице администратора.

Осталось сделать еще одну вещь: нам нужно сообщить администратору, что объекты Вопроса имеют интерфейс администратора. Для этого откройте файл __`polls/admin.py`__ и отредактируйте его так:

In [ ]:
# mysite/polls/admin.py 

from django.contrib import admin
from .models import Question

admin.site.register(Question)

# Изучите бесплатный функционал администратора

Теперь, когда мы зарегистрировали __`Question`__, Django знает, что он должен отображаться на индексной странице администратора:

Нажмите __`Question`__. Теперь вы находитесь на странице «списка изменений», где есть вопросы. На этой странице отображаются все вопросы в базе данных, и вы можете выбрать один из них, чтобы изменить его. Есть вопрос «Что случилось?» (“What’s up?”)  вопрос, который мы создали ранее:

Нажмите «Что случилось?» (“What’s up?”) вопрос по редактированию:

Здесь следует отметить:

- Форма автоматически генерируется на основе модели вопроса.
- Различные типы полей модели __`(DateTimeField, CharField)`__ соответствуют соответствующему виджету ввода HTML. Каждый тип поля знает, как отображаться в администраторе Django.
- Каждый __`DateTimeField`__ получает бесплатные ярлыки JavaScript. Даты получают ярлык «Сегодня» и всплывающее окно календаря, а время — ярлык «Сейчас» и удобное всплывающее окно со списком часто вводимых значений времени.

В нижней части страницы есть несколько вариантов:

- `save` — сохраняет изменения и возвращает на страницу списка изменений для объекта этого типа.
- `Save and continue editing` — сохраняет изменения и перезагружает страницу администратора для этого объекта.
- `Save and add another` — сохраняет изменения и загружает новую пустую форму для объекта этого типа.
- `Delete` – отображает страницу подтверждения удаления.

Если значение «Дата публикации» не соответствует времени, когда вы создали вопрос в уроке 1, это, вероятно, означает, что вы забыли установить правильное значение для параметра __`TIME_ZONE`__. Измените его, перезагрузите страницу и проверьте, отображается ли правильное значение.

Измените «Дата публикации», щелкнув ярлыки «Сегодня» и «Сейчас». Затем нажмите «Сохранить и продолжить редактирование». Затем нажмите «История» в правом верхнем углу. Вы увидите страницу со списком всех изменений, внесенных в этот объект через администратора Django, с отметкой времени и именем пользователя человека, который внес изменения:

Когда вы освоитесь с API моделей и ознакомитесь с сайтом администратора, прочитайте часть 3 этого руководства, чтобы узнать, как добавить больше представлений в наше приложение для опросов.

# RESUME:

16. `$ python manage.py migrate` -> создать таблицы в базе данных, прежде чем мы сможем их использовать.
17. `mysite/polls/models.py` -> в нашем приложении для опроса мы создадим две модели
18. `mysite/mysite/settings.py` -> Чтобы включить приложение в наш проект, нам нужно добавить ссылку на его класс конфигурации в параметр INSTALLED_APPS
19. `$ python manage.py makemigrations polls` -> вы сообщаете Django, что внесли некоторые изменения в свои модели (в данном случае вы внесли новые) и хотите, чтобы эти изменения были сохранены в виде миграции
20. `$ python manage.py sqlmigrate polls 0001` -> давайте посмотрим, какой SQL будет выполняться при этой миграции. Команда sqlmigrate принимает имена миграции и возвращает их SQL
21. `$ python manage.py migrate` -> Теперь снова запустите миграцию, чтобы создать эти таблицы моделей в вашей базе данных
22. `$ python manage.py shell` -> вызвать оболочку Python, используйте следующую команду
23. `>>> from polls.models import Choice, Question` -> импортируем 2 модели в оболочку пайтон
24. `>>> Question.objects.all()` -> смотрим все объекты класса Question 
25. `>>> from django.utils import timezone` -> Импортируем модули и библиотеки. Django ожидает дату и время с помощью tzinfo для pub_date. Используйте timezone.now()
26. `>>> q = Question(question_text="What's new?", pub_date=timezone.now())` -> создание нового вопроса "What's new?"
27. `>>> q.save()` -> сохроняем объект в базу данных
28. `>>> q.id` -> посмотрим идентификатор созданного объекта. (вопрос - есть объект)
29. `>>> q.question_text` -> Доступ к значениям поля question_text модели через атрибуты Python
30. `>>> q.pub_date` -> Доступ к значениям поля pub_date модели через атрибуты Python
31. `>>> q.question_text = "What's up?"` -> Измените значения, изменив атрибуты
32. `>>> q.save()` -> сохраняем изменения
33. `>>> Question.objects.all()` -> отображает все объекты в базе данных.
34. `mysite/polls/models.py` -> редактируем файл с моделями и добавив метод __str__()
35. `mysite/polls/models.py` -> Давайте также добавим в эту модель собственный метод was_published_recently
36. `>>> quit()` -> выход из интерактивной оболочки
37. `$ python manage.py shell` -> запускаем интерактивную оболочку после изменений и сохранений _mysite/polls/models.py_
38. `>>> from polls.models import Choice, Question` -> импортируем 2 модели в оболочку пайтон
39. `>>> Question.objects.all()` -> Убедитесь, что добавление `__str__()` сработало.
40. `>>> Question.objects.filter(id=1)` -> богатый API поиска в базе данных. ищем по ай ди.
41. `>>> Question.objects.filter(question_text__startswith="What")` -> ищем вопрос который начинается с "What"
42. `>>> from django.utils import timezone` -> импорты
43. `>>> current_year = timezone.now().year` -> выбираем год 
44. `>>> Question.objects.get(pub_date__year=current_year)` -> Получите вопрос, опубликованный в этом году.
45. `>>> Question.objects.get(id=2)` -> Запросите несуществующий идентификатор, это вызовет исключение.
46. `>>> Question.objects.get(pk=1)` -> Поиск по первичному ключу 
47. `>>> q = Question.objects.get(pk=1)` -> объект с праймери кей 1
48. `>>> q.was_published_recently()` -> Убедитесь, что наш собственный метод работает.
49. `>>> q = Question.objects.get(pk=1)` -> Дайте вопросу несколько вариантов выбора
50. `>>> q.choice_set.all()` -> отображение любых вариантов из набора связанных объектов
51. `>>> q.choice_set.create(choice_text="Not much", votes=0)`-> первый вариант выбора
52. `>>> q.choice_set.create(choice_text="The sky", votes=0)` -> второй вариант выбора
53. `>>> c = q.choice_set.create(choice_text="Just hacking again", votes=0)` -> третий вариант выбора поместили в переменную
54. `>>> c.question` -> Объекты выбора имеют доступ API к связанным с ними объектам вопросов.
55. `>>> q.choice_set.all()` -> объекты вопросов получают доступ к объектам выбора.
56. `>>> q.choice_set.count()` -> объекты вопросов получают доступ к объектам выбора.
57. `>>> Choice.objects.filter(question__pub_date__year=current_year)` -> Используйте двойное подчеркивание для разделения отношений. Это работает на любом уровне глубины
58. `>>> c = q.choice_set.filter(choice_text__startswith="Just hacking")` -> Давайте удалим один из вариантов. Используйте для этого delete().
59. `>>> c.delete()`
60. `$ python manage.py createsuperuser` -> создать пользователя, который сможет войти на сайт администратора. вводим имя пользователя, емейл и пароль. 
61. `$ python manage.py runserver` -> запускаем сервер разработки.
62. `mysite/polls/admin.py` -> нам нужно сообщить администратору, что объекты Вопроса имеют интерфейс администратора
63. `http://127.0.0.1:8000/admin/` -> Индексная страница администратора